<h1> Project Progress </h1>

This file will track the progress of this project, including major steps, code changes, results, etc. 
A ipynb file is best as it will allow for code, writing, etc and best formatting. 

<h1> Baseline: Stage 1 <h1>

Full Observability
Hold time: 3 step
No extraction
No ghost movement

<h2> VDN </h2>
Did not work. This was the last and best attempt

[INFO 10:16:01] my_main Recent Stats | t_env:     199894 | Episode:      676
ep_length_mean:          300.0000       epsilon:                   0.0500       grad_norm:               2968.1982      hold_mean:                 0.0000
loss:                    9660.9391      q_taken_mean:              4.3232       return_mean:             462.8749       return_std:              980.8482
success_mean:              0.0000       t_mean:                  300.0000       target_mean:               5.4364       td_error_abs:             10.1837
test_ep_length_mean:     286.3500       test_hold_mean:            0.2500       test_return_mean:        841.0543       test_return_std:         1298.6543
test_success_mean:         0.0500       test_t_mean:             286.3500
[INFO 10:16:01] my_main Finished Training

In [12]:
Image(url="Baseline_2_BeforeQMIX.png", width=900, height=600)

<h2> QMIX</h2>
Second attempt at baseline

First Attempt:

ep_length_mean:          296.0036       epsilon:                   0.0500       grad_norm:               44408.9617     hold_mean:                 0.2679
loss:                    15895.5070     q_taken_mean:              8.6338       return_mean:             1068.4453      return_std:              1801.3856
success_mean:              0.0536       t_mean:                  296.0036       target_mean:               8.5500       td_error_abs:             21.6396
test_ep_length_mean:     262.0500       test_hold_mean:            0.7500       test_return_mean:        2047.0497      test_return_std:         2918.8472
test_success_mean:         0.1500       test_t_mean:             262.0500

In [7]:
# import image module
from IPython.display import Image

# get the image
Image(url="BaselineFirstQMIX.png", width=900, height=600)

I notice that gradient is EXPLODING and loss is huge. I change my reward structure to make it smoother and more normalized

Second Attempt:
ep_length_mean:          250.6667       epsilon:                   0.0500       grad_norm:                 1.3428       hold_mean:                 1.2222
loss:                      0.7356       q_taken_mean:              0.2904       return_mean:             -248.2167      return_std:              520.8183
success_mean:              0.2444       t_mean:                  250.6667       target_mean:               0.2837       td_error_abs:              0.1454
test_ep_length_mean:     271.5500       test_hold_mean:            0.7500       test_return_mean:        -160.0374      test_return_std:         573.6516

In [8]:
Image(url="QMIX2ndBaseline.png", width=900, height=600)

Clearly doing much better and trending in correct direction. 
We see that QMIX is doing much better, and a better reward structure is paying off. 

In [ ]:
#Make distance shaping even better : exponential pull towards ghost    def AgentToGhostDist_Reward(self, prev_agent_ghost_d):

def AgentToGhostDist_Reward(self, prev_agent_ghost_d):
    reward = 0.0

    # Closer to ghost = higher reward every step
    max_dist = max(self.grid.width, self.grid.height) - 1
    
    for a in self.agents:
        dist = cheb_dist((a.x, a.y), self.ghostCoords)
        norm_dist = dist / max_dist
        reward += self.lambda_agent_dist_to_ghost * ((1.0 - norm_dist) ** 2) * 2.0 #exponential pull for stronger incentive 

    return reward

<h2> BOOM here we are </h2>

[INFO 11:05:27] my_main Recent Stats | t_env:     140057 | Episode:     1484
ep_length_mean:           30.6744       epsilon:                   0.3431       grad_norm:                31.0257       hold_mean:                 5.0000
loss:                      1.9014       q_taken_mean:              2.5236       return_mean:             320.9782       return_std:               71.2775
success_mean:              1.0000       t_mean:                   30.6744       target_mean:               2.4942       td_error_abs:              0.7269
test_ep_length_mean:      32.1500       test_hold_mean:            5.0000       test_return_mean:        365.2984       test_return_std:         108.3838
test_success_mean:         1.0000       test_t_mean:              32.1500


Exponential shaping made a huuuuuuge difference

Proof that VDN could not do this: 
Same env, rewards, etc, just with vdn

[INFO 11:09:27] my_main Recent Stats | t_env:     198294 | Episode:      672
ep_length_mean:          295.8000       epsilon:                   0.0500       grad_norm:               112.0349       hold_mean:                 0.1250
loss:                     10.1877       q_taken_mean:              3.3301       return_mean:             762.8890       return_std:              271.7277
success_mean:              0.0250       t_mean:                  295.8000       target_mean:               3.3562       td_error_abs:              1.4568
test_ep_length_mean:     300.0000       test_hold_mean:            0.0000       test_return_mean:        897.3207       test_return_std:         192.5651
test_success_mean:         0.0000       test_t_mean:             300.0000

<h1>Stage 2: Capturing a moving ghost</h1>

Ghost now moves 30% of the time, with a 20% decrease for every agent near it
Spawns further away : 15

First attempt gives 35% success rate. Its ok but not great. We want this to improve
We test the policy (QMIX_Stage2_1_) and notice that they quickly get to it and close to it, but can struggle to fully surround and capture.
I notice ghosts tend to enter surround and leave again and again, so I will strongly reward staying in surround radius, and decay the bonus for a new complete surround.
Also will encourage coverage on multiple directions/quadrants of the ghost

In [13]:
def globalReward(self, prev_d_ghost_extract, prev_d_ghost_agents, prev_surround_count):
        reward = -0.01  # Tiny time penalty to encourage efficiency (won't dominate)
        terminated = False
        success = False

        if self.surroundCounter >= numAgents:
            self.holdCounter += 1
        else:
            self.holdCounter = 0

        # End conditions
        if self.holdCounter >= self.timeToKill:
            reward += self.win_reward  # HUGE success reward to make it unmissable
            terminated = True
            success = True
        elif self.Time >= self.episodeLimit:
            terminated = True
            # NO massive failure penalty - let shaped rewards guide learning

        if terminated:
            return reward, terminated, success
        

        # Reward for getting agents closer to ghost
        reward += self.AgentToGhostDist_Reward(prev_d_ghost_agents)

        #Reward for surrounding ghost
        reward += self.Surround_Reward(prev_surround_count)
        
        # Reward for spreading around ghost at different angles
        if self.surroundCounter >= 2:  # Only when at least 2 agents nearby
            reward += self.QuadrantCoverage_Reward()

        #penalty for agents too close to each other (when NOT surrounding ghost)
        if self.surroundCounter < numAgents:
            pass
            reward += self.AgentClumpingPenalty()


        return reward, terminated, success


def QuadrantCoverage_Reward(self):
        """
        Reward agents for surrounding ghost from different cardinal directions.
        Checks if agents are positioned left, right, above, or below the ghost.
        Rewards for each direction covered.
        """
        
        gx, gy = self.ghostCoords
        directions = [False, False, False, False]  # Left, Right, Above, Below
        
        for ax, ay in self.agentCoords:
            # Only count agents within surround radius
            if cheb_dist((ax, ay), (gx, gy)) <= self.surroundRadius:
                dx = ax - gx
                dy = ay - gy
                
                # Determine primary direction (using dominance of dx vs dy)
                if abs(dx) > abs(dy):  # Horizontal dominance
                    if dx > 0:
                        directions[1] = True  # Right
                    else:
                        directions[0] = True  # Left
                else:  # Vertical dominance
                    if dy > 0:
                        directions[2] = True  # Above
                    else:
                        directions[3] = True  # Below
        
        # Reward per direction covered
        num_covered = sum(directions)
        return self.lambda_quadrant_coverage * num_covered

From visual testing what is happening is that they get close very quickly, but struggle to fully surround and capture. It seems like agents get in the way, and possibly exploit coming in and out of surround radius

ALSO: Results great until 200k steps, then things break. Seems like issues with epsilon: exploration etc. 

In [14]:
Image(url="QMIX_Stage2_AfterQuadrantsAndDecay.png", width=900, height=600)

At this point I decided to try stage 3 and basically same rewards and same behaviour. 
I notice once agents get close to ghost, they behave nearly identically. 

So I'm going to try enabling adding agent id to observations to allow for more heterogenous actions and more "separation" between the agents. 
I Cubed the distance so they wanna get even closer
Then I made the avoid radius smaller

NOTHING was working. So I had to dig deeper to analyze and given their erratic behaviour, was suspucious of an underlying problem so I looked at step how movement was working. This lead me to step(). 

"

In [ ]:
#Before

# 1. Agents act
for a, act_id in zip(self.agents, actions):
    a.apply_action(act_id)

# 2. Ghost moves (AFTER agents)
self.ghostCoords = self.ghost.move(self.agentCoords)

# 3. Calculate rewards based on NEW positions
reward = self.globalReward(...)

In [ ]:
# 1. Ghost moves FIRST
self.ghostCoords = self.ghost.move(self.agentCoords)

# 2. Agents act (seeing where ghost went)
for a, act_id in zip(self.agents, actions):
    a.apply_action(act_id)

# 3. Calculate rewards
reward = self.globalReward(...)

In [ ]:
Image(url="QMIX_Stage3_UpdatedStep.png", width=900, height=600)

In [ ]:
    def AgentToGhostDist_Reward(self, prev_agent_ghost_d):
        reward = 0.0

        # Closer to ghost = higher reward every step
        # Use Chebyshev distance to match surround logic!
        max_dist = max(self.grid.width - 1, self.grid.height - 1)
        
        for a in self.agents:
            dist = cheb_dist((a.x, a.y), self.ghostCoords)
            norm_dist = dist / max_dist
            # STRONGER exponential: cube instead of square for aggressive pull when close
            reward += self.lambda_agent_dist_to_ghost * ((1.0 - norm_dist) ** 3) * 3.0

        return reward
    
    def Surround_Reward(self, prev_surround_count):
        reward = 0.0
        new_surround_count = self.ghost.GetSurroundedCount(self.agentCoords)

        # Progressive surround rewards - BIG bonus for entering radius
        if new_surround_count > prev_surround_count:
            reward += self.reward_new_surround * (new_surround_count - prev_surround_count) * 2.0
        
        # PENALTY: Breaking formation (agents leaving surround) - REDUCED to encourage commitment
        if new_surround_count < prev_surround_count:
            penalty = self.penalty_formation_break * (prev_surround_count - new_surround_count) * 0.5
            reward -= penalty
        
        
        # Keeping full surround
        if new_surround_count >= numAgents:
            reward += self.reward_full_surround

        return reward

<h1> Stage 4: Extraction <h1>
We change the game: now must extract.

First attempts fail. 

Not going well. 


In [ ]:

        #For shaping rewards per phase of game
        surround_progress = self.surroundCounter / numAgents  # 0.0 → 1.0
        
        extraction_multiplier = 0.1 + 1.9 * (surround_progress ** 3)
        surrounding_multiplier = 1.0

        #STEP1: Surrounding
        reward += surrounding_multiplier * self.AgentToGhostDist_Reward(prev_d_ghost_agents)
        reward += surrounding_multiplier * self.Surround_Reward(prev_surround_count)
        reward += surrounding_multiplier * self.QuadrantCoverage_Reward()

        #Step2: Extraction
        reward += extraction_multiplier * self.GhostToExtraction_Reward(prev_d_ghost_extract)

        return reward, terminated, success

Now I will heavily reward not (only) surrounding, but leaving a hole aka pushing on the correct side. 

so we would have to computer the required direction for ghost to go, then reward the agent's position for aligning with this, but from the other direciotn

I'm thiking we could find the general line of the agents, and take the dot product with the ideal one. This tells us how close we are to that, and we can reward this. 